<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Análise-Exploratória-de-Dados" data-toc-modified-id="Análise-Exploratória-de-Dados-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Análise Exploratória de Dados</a></span><ul class="toc-item"><li><span><a href="#Séries-temporais" data-toc-modified-id="Séries-temporais-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Séries temporais</a></span></li><li><span><a href="#Relação-entre-duas-variáveis-continuas" data-toc-modified-id="Relação-entre-duas-variáveis-continuas-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Relação entre duas variáveis continuas</a></span></li><li><span><a href="#Relação-entre-uma-variável-continua-e-uma-categórica" data-toc-modified-id="Relação-entre-uma-variável-continua-e-uma-categórica-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Relação entre uma variável continua e uma categórica</a></span></li><li><span><a href="#Relação-entre-múltiplas-variáveis-continuas" data-toc-modified-id="Relação-entre-múltiplas-variáveis-continuas-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Relação entre múltiplas variáveis continuas</a></span></li></ul></li></ul></div>

In [ ]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv("credentials/mysql.env")
url_banco = "localhost"
nome_db = "olist"
conn_str = f"mysql+pymysql://{os.getenv('MYSQL_USER')}:{os.getenv('MYSQL_PASS')}@{url_banco}/{nome_db}"
engine = create_engine(conn_str)


In [ ]:
sns.set_theme(context="notebook", style="darkgrid")


Para facilitar nosso trabalho na aula de hoje, vamos criar um View com os dados que utilizaremos ao longo da aula.

# Análise Exploratória de Dados (EDA)

O processo de construção de modelos pode ser dividido em quatro etapas:

1. **Definição do Problema e Coleta de Dados**;
    * Todo modelo deve ter um **objetivo bem definido**.
    * O **tempo disponível** para construção é **parte da definição do problema**!
    * A partir desta definição podemos definir *quais dados serão necessários* para **iniciarmos nossa análise**.
1. **Análises Exploratórias**;
    * O objetivo da análise exploratória é *maximizar o nosso conhecimento* sobre a **estrutura dos dados** disponíveis, mapear as relações entre diferentes variáveis e o objetivo do nosso problema e **avaliar a qualidade dos dados**, eventualmente propondo tratativas para sanar problemas.
1. **Definição e construção do modelo**;
    * A partir do conhecimento adquirido durante a análise exploratória vamos determinar quais **técnicas são mais apropriadas** para a resolução do nosso problema.
    * Para escolher entre diferentes técnicas devemos **implantar uma infra-estrutura rudimentar de testes**, que nos permita *testar diferentes modelos de forma rápida e simples*.
1. **Validação e definição de próximos passos**.
    * Com a construção do modelo finalizada, conduziremos a validação final do modelo, apresentando os conceitos por trás deste para os diferentes stakeholders do projeto. Além disso conduziremos testes utilizando a infra-estrutura operacional do modelo, garantindo que não houve perda de performance nas condições operacionais.

Este processo **não é necessariamente linear**: podemos descobrir na etapa de análises exploratórias que não temos todas as informações necessárias para construir um modelo, ou então que as informações estão comprometidas a ponto de inviabilizar qualquer análise.

Hoje nos aprofundaremos na segunda etapa deste fluxo: a análise exploratória de dados!

## Definição do Problema

Para a aula de hoje utilizaremos o data set da Olist. Foi solicitado ao CoE de Dados que construa um modelo para explicar e prever variações nas avalições que clientes fizeram de nossos pedidos. O objetivo deste modelo é descobrir quais são os principais eixos que impactam estas avaliações para que possamos focar esforços na melhoria das áreas mais críticas.

Os dados utilizados serão os dados do case OLIST, disponíveis no nosso DB MySQL.

## Fonte de Dados

No caso da aula de hoje, utilizaremos o nosso DB MySQL como fonte de dados para nossa análise exploratória.

In [ ]:
query = """
	SELECT
		oodc.order_id,
		ooidc.seller_id,
		ooidc.price,
		ooidc.freight_value,
		oodc.order_status,
		STR_TO_DATE(oodc.order_approved_at, '%%Y-%%m-%%d %%H:%%i:%%s') AS order_approved_at,
		STR_TO_DATE(ooidc.shipping_limit_date, '%%Y-%%m-%%d %%H:%%i:%%s') AS shipping_limit_date,
		STR_TO_DATE(oodc.order_delivered_carrier_date, '%%Y-%%m-%%d %%H:%%i:%%s') AS order_delivered_carrier_date,
		STR_TO_DATE(oodc.order_delivered_customer_date, '%%Y-%%m-%%d %%H:%%i:%%s') AS order_delivered_customer_date,
		STR_TO_DATE(oodc.order_estimated_delivery_date, '%%Y-%%m-%%d %%H:%%i:%%s') AS order_estimated_delivery_date,
		ocdc.customer_state,
		osdc.seller_state,
		opdc.product_category_name,
		opdc.product_weight_g,
		oordc.review_score 
	FROM 
		olist_order_items_dataset_csv ooidc JOIN
		olist_orders_dataset_csv oodc ON (ooidc.order_id = oodc.order_id) JOIN 
		olist_customers_dataset_csv ocdc ON (oodc.customer_id = ocdc.customer_id) JOIN 
		olist_products_dataset_csv opdc ON (ooidc.product_id = opdc.product_id) JOIN
		olist_sellers_dataset_csv osdc ON (osdc.seller_id = ooidc.seller_id) LEFT JOIN 
		olist_order_reviews_dataset_csv oordc ON (oodc.order_id = oordc.order_id) JOIN
		(
			SELECT
				oordc.review_id,
				STR_TO_DATE(oordc.review_creation_date, '%%Y-%%m-%%d %%H:%%i:%%s') AS review_date,
				SUM(1) OVER (PARTITION BY order_id ORDER BY order_id, STR_TO_DATE(oordc.review_creation_date, '%%Y-%%m-%%d %%H:%%i:%%s') DESC) AS cum_review
			FROM 
				olist_order_reviews_dataset_csv oordc
			ORDER BY 
				oordc.order_id,
				STR_TO_DATE(oordc.review_creation_date, '%%Y-%%m-%%d %%H:%%i:%%s') DESC
		) AS lr ON (oordc.review_id = lr.review_id)
	WHERE
		lr.cum_review = 1
	HAVING
		order_approved_at >= '2017-08-01' AND
		order_approved_at < '2018-08-01'
"""
tb_pedidos = pd.read_sql(query, engine)


In [ ]:
tb_pedidos.info()


In [ ]:
tb_pedidos.describe()


In [ ]:
tb_pedidos.select_dtypes(include=["object"]).describe()


In [ ]:
tb_pedidos.select_dtypes(include=["datetime"]).describe(datetime_is_numeric=True)


In [ ]:
colunas_data = tb_pedidos.select_dtypes(include=["datetime"]).columns

for coluna in colunas_data:
    tb_pedidos[coluna] = tb_pedidos[coluna].dt.normalize()

tb_pedidos["atraso"] = (
    tb_pedidos["order_delivered_customer_date"]
    > tb_pedidos["order_estimated_delivery_date"]
)

tb_pedidos.loc[tb_pedidos["atraso"], "dias_atraso"] = (
    tb_pedidos["order_delivered_customer_date"]
    - tb_pedidos["order_estimated_delivery_date"]
) / np.timedelta64(1, "D")
tb_pedidos.loc[~tb_pedidos["atraso"], "dias_atraso"] = 0


In [ ]:
tb_pedidos = tb_pedidos.dropna()
tb_pedidos.info()


## Usando Paletas de Cor

Um dos principais elementos na gramática dos gráficos é a cor. Podemos utilizar a cor de forma gradual (através de **paletas uniformes ou divergentes**) para representar variáveis continuas ou então de forma discreta (através de **paletas qualitativas**).

A biblioteca `seaborn` vem com algumas paletas de cores pré-configuradas, as quais podemos investigar e acessar através da função `color_palette()`. Esta função nos permite visualizar os elementos discretos de uma paleta através de seu nome, e acessar cada cor especificamente através de sua indexação.

In [ ]:
sns.color_palette("Paired")


Para padronizar a utilização de cores podemos guardar paletas em variáveis para utilização futura:

In [ ]:
colors_discrete = sns.color_palette("Paired")


Vamos criar um subset desta paleta, mantendo apenas as cores fortes:

In [ ]:
colors_discrete_strong = [color for color in colors_discrete if colors_discrete.index(color) % 2 != 0]

In [ ]:
sns.color_palette("icefire", as_cmap=True)


In [ ]:
colors_cont = sns.color_palette("icefire", as_cmap=True)


## Distribuição e Evolução de Avaliações

Vamos inicializar nossa análise investigando a distribuição e evolução histórica das avaliações.

In [ ]:
sns.histplot(data=tb_pedidos, x="review_score")


Como as notas dadas pelos clientes são discretas (de 1 à 5 estrelas provavelmente), nosso histograma se parece mais com um barplot. Para facilitar a leitura vamos utilizar um barplot diretamente:

In [ ]:
sns.countplot(data=tb_pedidos, x="review_score", palette = colors_discrete_strong)

Podemos ver que existe uma concentração de avaliações nos valores extremos (1 e 5) - vamos agrupar essas notas em duas categorias, marcando as notas 1, 2 e 3 como `detractor`.

In [ ]:
tb_pedidos["detractor"] = np.where(tb_pedidos["review_score"] <= 3, 1, 0)


Para visualizar a evolução histórica das avaliações precisamos agrupar os pedidos (granularidade do nosso `DataFrame`) em **time buckets**, intervalos de tempo fixos. Vamos utilizar **time buckets** diários para esta análise.

In [ ]:
tb_diaria = (
    tb_pedidos.groupby("order_approved_at")
    .agg(
        num_pedidos=pd.NamedAgg("order_id", "nunique"),
        avg_review=pd.NamedAgg("review_score", "mean"),
        std_review=pd.NamedAgg("review_score", "std"),
        per_detractor=pd.NamedAgg("detractor", "mean"),
        per_atraso=pd.NamedAgg("atraso", "mean"),
        avg_atraso=pd.NamedAgg("dias_atraso", "mean"),
    )
    .reset_index()
)


Vamos utilizar diferentes gráficos de linha para avaliar a evolução do número de pedidos, atrasos e avaliações:

In [ ]:
plt.figure(figsize=(12, 4))
sns.lineplot(data=tb_diaria, x="order_approved_at", y="num_pedidos")


In [ ]:
plt.figure(figsize=(12, 4))
sns.lineplot(data=tb_diaria, x="order_approved_at", y="avg_review")


In [ ]:
plt.figure(figsize=(12, 4))
sns.lineplot(data=tb_diaria, x="order_approved_at", y="per_detractor")


In [ ]:
tb_diaria["cv_review"] = tb_diaria["std_review"] / tb_diaria["avg_review"]

plt.figure(figsize=(12, 4))
sns.lineplot(data=tb_diaria, x="order_approved_at", y="cv_review")


In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(12, 16))

sns.lineplot(data=tb_diaria, x="order_approved_at", y="num_pedidos", ax=ax[0])
sns.lineplot(data=tb_diaria, x="order_approved_at", y="avg_review", ax=ax[1])
sns.lineplot(data=tb_diaria, x="order_approved_at", y="cv_review", ax=ax[2])
sns.lineplot(data=tb_diaria, x="order_approved_at", y="per_detractor", ax=ax[3])


Embora a análise das séries diárias nos permita ver fenômenos como a sazonalidade de dia da semana e eventuais outliers, a alta frequência da série pode ocultar padrões mais estáveis. Vamos suaziar estas séries utilizando médias móveis:

In [ ]:
tb_diaria["mm_num_pedidos"] = tb_diaria["num_pedidos"].rolling(7).mean()
tb_diaria["mm_avg_review"] = tb_diaria["avg_review"].rolling(7).mean()
tb_diaria["mm_cv_review"] = tb_diaria["cv_review"].rolling(7).mean()
tb_diaria["mm_per_detractor"] = tb_diaria["per_detractor"].rolling(7).mean()
tb_diaria["mm_per_atraso"] = tb_diaria["per_atraso"].rolling(7).mean()
tb_diaria.head(10)


Agora vamos combinar as visualizações que construímos anteriormente (da visão diária) com as novas variáveis que construímos (médias móveis):

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(9, 12))

sns.scatterplot(
    data=tb_diaria,
    x="order_approved_at",
    y="num_pedidos",
    color=colors_discrete[0],
    ax=ax[0],
    alpha=0.9,
)
sns.lineplot(
    data=tb_diaria,
    x="order_approved_at",
    y="mm_num_pedidos",
    color=colors_discrete[1],
    ax=ax[0],
)
sns.scatterplot(
    data=tb_diaria,
    x="order_approved_at",
    y="avg_review",
    color=colors_discrete[2],
    ax=ax[1],
    alpha=0.9,
)
sns.lineplot(
    data=tb_diaria,
    x="order_approved_at",
    y="mm_avg_review",
    color=colors_discrete[3],
    ax=ax[1],
)
sns.scatterplot(
    data=tb_diaria,
    x="order_approved_at",
    y="cv_review",
    color=colors_discrete[4],
    ax=ax[2],
    alpha=0.9,
)
sns.lineplot(
    data=tb_diaria,
    x="order_approved_at",
    y="mm_cv_review",
    color=colors_discrete[5],
    ax=ax[2],
)
sns.scatterplot(
    data=tb_diaria,
    x="order_approved_at",
    y="per_detractor",
    color=colors_discrete[6],
    ax=ax[3],
    alpha=0.9,
)
sns.lineplot(
    data=tb_diaria,
    x="order_approved_at",
    y="mm_per_detractor",
    color=colors_discrete[7],
    ax=ax[3],
)
sns.scatterplot(
    data=tb_diaria,
    x="order_approved_at",
    y="per_atraso",
    color=colors_discrete[8],
    ax=ax[4],
    alpha=0.9,
)
sns.lineplot(
    data=tb_diaria,
    x="order_approved_at",
    y="mm_per_atraso",
    color=colors_discrete[9],
    ax=ax[4],
)


## Explorando (co)Relações

Através da **análise de séries temporais** conseguimos encontrar correlações temporais (no nível diário, não do pedido) entre alguns indicadores agregados. Vamos avaliar essas relações mais de perto utilizando *scatterplots* para **visualizar diretamente a estrutura de correlação** entre nossas variáveis temporais.

In [ ]:
sns.scatterplot(data=tb_diaria, x="num_pedidos", y="avg_review")


In [ ]:
sns.scatterplot(data=tb_diaria, x="mm_num_pedidos", y="mm_avg_review")


Vamos utilizar o elemento cor para representar a taxa de atraso, permitindo visualizarmos a relação entre 3 variáveis diretamente:

In [ ]:
sns.scatterplot(data=tb_diaria, x="mm_num_pedidos", y="mm_avg_review", hue="per_atraso")


Uma deficiência dos scatterplots é o **empilhamento de pontos**, que muitas vezes dificulta a visualização de regiões com maior densidade de pontos. Vamos utilizar a função `kdeplot()` para representar curvas de nível da densidade de pontos.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 5))
sns.kdeplot(data=tb_diaria, x="mm_num_pedidos", y="mm_avg_review", fill = True, ax = ax[0])
sns.kdeplot(data=tb_diaria, x="mm_per_atraso", y="mm_avg_review", fill = True, ax = ax[1])

## Relação entre uma variável continua e uma categórica

Agora vamos retornar à nossa base original, no nível linha do pedido, para analisarmos quais fatores discretos impactam a **% de atrasos** observada.

In [ ]:
tb_pedidos = tb_pedidos.merge(
    tb_diaria[["order_approved_at", "num_pedidos"]], on="order_approved_at"
)


Vamos começar analisando a distribuição de peso dos itens comprados pela categorização de avaliação que fizemos:

In [ ]:
sns.boxplot(
    data=tb_pedidos,
    x="detractor",
    y="product_weight_g",
)


Que gráfico horrível! Sempre que observamos uma variável (estritamente positiva) cuja distribuição está muito concentrada em valores baixos mas com muitos outliers, podemos considerar uma transformação logaritmíca para facilitar a visualização dos resultados:

In [ ]:
g = sns.boxplot(
    data=tb_pedidos[tb_pedidos["product_weight_g"] > 0],
    x="detractor",
    y="product_weight_g",
)
g.set_yscale("log")


Parece que o peso do item comprado não tem impacto sobre a avaliação... Testar variável a variável desta forma pode ser trabalhoso e nem sempre renderá avanços no nosso modelo. Vamos utilizar uma técnica reminescente das regressões lineares para facilitar nossa exploração: a **análise de resíduos (ou erros)**.

In [ ]:
g = sns.boxplot(data=tb_pedidos, x="atraso", y="review_score");


Claramente o *status* de atraso tem um impacto enorme nas avaliações, mas isso já era um fato conhecido! No entanto temos pedidos com avaliações <= 3 sem que houvesse atraso. Vamos criar uma base filtrada para buscar outros fatores que podem impactar a avaliação do cliente:

In [ ]:
mask_out = (~tb_pedidos["atraso"]) & (tb_pedidos["review_score"] <= 3)
tb_pedidos_out = tb_pedidos[mask_out].copy()


Além do atraso, um eixo comum de reclamação é a **qualidade dos produtos** pedidos. Vamos descobrir se existem categorias ofensoras dentro do nosso grupo de pedidos mal-avaliados sem atraso:

In [ ]:
sns.countplot(data=tb_pedidos_out, x="product_category_name");


O que está errado com esse gráfico?

In [ ]:
tb_cat_out = (
    tb_pedidos_out.groupby("product_category_name")["order_id"]
    .count()
    .reset_index()
    .rename({"order_id": "num_pedidos_out"}, axis=1)
)

tb_cat = (
    tb_pedidos.groupby("product_category_name")["order_id"]
    .count()
    .reset_index()
    .rename({"order_id": "num_pedidos_total"}, axis=1)
)

tb_cat = tb_cat.merge(tb_cat_out, on="product_category_name")
tb_cat["prop_total"] = tb_cat["num_pedidos_total"] / sum(tb_cat["num_pedidos_total"])
tb_cat["num_pedidos_out_expct"] = tb_cat["prop_total"] * sum(tb_cat["num_pedidos_out"])
tb_cat["diff_pedidos_out"] = tb_cat["num_pedidos_out"] - tb_cat["num_pedidos_out_expct"]


Agora podemos visualizar se existem categorias cujo # de pedidos dentro do grupo selecionado (sem atraso) é maior do que o esperado:

In [ ]:
sns.histplot(data=tb_cat, x="diff_pedidos_out")


Temos algumas ofensoras claras! Vamos filtrar a tabela acima e selecionar estas categorias:

In [ ]:
cat_problema = set(
    tb_cat.loc[tb_cat["diff_pedidos_out"] > 100, "product_category_name"]
)
cat_problema


Agora vamos utilizar esse conjunto de categorias para realizar um agrupamento da variável categoria na base original de pedidos:

In [ ]:
mask_cat = tb_pedidos["product_category_name"].map(lambda x: x in cat_problema)
tb_pedidos.loc[mask_cat, "cat_agg"] = tb_pedidos["product_category_name"]
tb_pedidos.loc[~mask_cat, "cat_agg"] = "Outros"
fig, ax = plt.subplots(figsize = (10, 5))
sns.boxplot(data=tb_pedidos, x="atraso", y="review_score", hue="cat_agg")


A natureza discreta do campo de avaliação dificulta a visualização da distribuição através de boxplots. Vamos utilizar um violin plot para aumentar a

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.violinplot(data=tb_pedidos, x="cat_agg", y="review_score", hue="atraso", split=True)


## Análises multi-variáveis

In [ ]:
tb_diaria.columns


In [ ]:
var_original = [
    "num_pedidos",
    "avg_review",
    "std_review",
    "per_detractor",
    "per_atraso",
    "avg_atraso",
]
var_mm = ["mm_num_pedidos", "mm_avg_review", "mm_cv_review", "mm_per_detractor"]


In [ ]:
sns.pairplot(tb_diaria[var_original])


In [ ]:
sns.pairplot(tb_diaria[var_mm])


In [ ]:
plt.figure(figsize=(12, 9))
sns.heatmap(tb_diaria[var_original].corr(), vmin=-1, center=0, vmax=1)


In [ ]:
plt.figure(figsize=(12, 9))
sns.clustermap(tb_diaria[var_mm].corr(), vmin=-1, center=0, vmax=1)
